# 4. 智能体评估与调试

## 4.1 简介
在生成式大语言模型的世界里，智能体就像是一个充满潜力的助手，它能够根据我们的指令完成各种任务。然而，就像任何新生事物一样，智能体在实际应用中可能会出现各种问题。为了确保智能体能够高效、准确地完成任务，我们需要对其进行评估和调试。本章将深入探讨智能体评估与调试的重要性、方法以及常见问题的处理方式。



## 准备工作
我们先按照前两节介绍的，部署一个生成式大模型作为AI智能体。我们将在这一章节里以代码的形式详细介绍如何评估该智能体。

In [1]:
import torch
# 确保 PyTorch 可以使用 GPU
assert torch.cuda.is_available()

from transformers import AutoTokenizer, AutoModelForCausalLM

# 指定模型名称或本地路径
model_name = "/root/models/Qwen2.5-0.5B-Instruct"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 将模型移动到 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded on cuda


查看模型的参数量非常重要，因为如果我们把智能体类比为人脑，模型参数大小就是人脑的大小。直观来说，越大的模型越聪明，效果越好。

In [19]:
# 查看模型参数量
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1e6:.2f}M")

# 查看模型结构
print(model)

Total model parameters: 494.03M
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )



## 4.2 为什么需要评估和调试智能体
智能体在执行复杂任务时，可能会出现诸多问题。比如在回答问题时，可能会给出错误的答案；在使用工具时，可能会失败；在进行推理时，可能会出现不当之处。这些问题可能会严重影响智能体的性能和用户体验。因此，评估智能体的表现是至关重要的。通过评估，我们可以了解智能体在不同任务中的表现，发现潜在的问题，并评估问题的严重性。同时，评估还能帮助我们确认优化方案的有效性，以便对智能体进行针对性的改进。
在实际应用中，智能体可能会出现一些常见的失败情况。以下举例几个典型的失败现象：





### 4.2.1 幻觉现象
即使生成流畅、专业的文本，智能体也可能给出虚假的事实或编造信息，这被称为“幻觉（hallucination）”。例如，OpenAI 的最新推理模型 o3 和 o4-mini 在 PersonQA 问答测试中的幻觉率分别高达 33% 和 48% 。而 Meta 的 Galactica 模型在生成科学论文时还引用了不存在的参考文献，最终导致其撤回。这类错误不仅影响用户体验，在医疗、法律、财经等高风险领域可能造成严重后果，例如错误推理病因，错误做出判决。


智能体像有幻觉一样，笃定地诉说或证明一件不正确的事。

例如，当你问智能体：“今年的美国总统是哪位？”
它回答：“奥巴马。”
你又问：“那谁发动了南北战争，解放了奴隶？”
它回答：“奥巴马作为第一任黑人总统，解放了所有黑奴。”


In [ ]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device=device  # 指定设备
)

prompt = \
"""
怪奇物语是谁写的
""" 

print(prompt)
# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=2000,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)


怪奇物语是谁写的



/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
怪奇物语是日本作家山本茂创作的中篇小说。该作品在1935年首次发表，在日本和欧美出版后，一直受到读者的喜爱。

## 作者信息

### 生平
山本茂（1874-1960）生于东京，毕业于东京帝国大学工学部，后于1923年成为东京帝国大学名誉教授，之后又任东京帝国大学文学院院长、东京帝国大学法学院院长等职。他的主要文学作品有《梦》、《幻影》、《神魔传》等。其中，《梦》是其代表作之一，以“怪奇物语”为题写成的长篇小说，以日本近代历史为基础，通过主人公的视角叙述了从明治维新到一战期间的历史事件。《梦》是日本第一部现代小说，并且被认为是日本第一部现代小说。

### 出版
1.1935年《怪奇物语》由山本茂首次发表，获第1届日本内政省文学奖。
1.1936年《怪奇物语》由山本茂再度出版，获第2届日本内政省文学奖。
1.1937年《怪奇物语》由山本茂再度出版，获第3届日本内政省文学奖。
1.1938年《怪奇物语》由山本茂再度出版，获第4届日本内政省文学奖。
1.1939年《怪奇物语》由山本茂再度出版，获第5届日本内政省文学奖。
1.1940年《怪奇物语》由山本茂再度出版，获第6届日本内政省文学奖。
1.1941年《怪奇物语》由山本茂再度出版，获第7届日本内政省文学奖。
1.1942年《怪奇物语》由山本茂再度出版，获第8届日本内政省文学奖。
1.1943年《怪奇物语》由山本茂再度出版，获第9届日本内政省文学奖。
1.1944年《怪奇物语》由山本茂再度出版，获第10届日本内政省文学奖。
1.1945年《怪奇物语》由山本茂再度出版，获第11届日本内政省文学奖。
1.1946年《怪奇物语》由山本茂再度出版，获第12届日本内政省文学奖。
1.1947年《怪奇物语》由山本茂再度出版，获第13届日本内政省文学奖。
1.1948年《怪奇物语》由山本茂再度出版，获第14届日本内政省文学奖。
1.1949年《怪奇物语》由山本茂再度出版，获第15届日本内政省文学奖。
1.1950年《怪奇物语》由山本茂再度出版，获第16届日本内政省文学奖。
1.1951年《怪奇物语》由山本茂再度出版，获第17届日本内政省文学奖。
1.1952年《怪奇物语》由山本茂再度出版，获第18届日本内政省文学奖。
1.1953年《怪奇物语》由山本茂再度出版，获第19届日本内政省文学奖。
1.1954年《怪奇物语

### 4.2.2 工具调用失败：
当智能体尝试调用外部 API 或插件时，可能出现语法格式错误、缺失必需参数、工具不存在或接口响应不一致的问题。论文《T‑Eval: Evaluating the Tool Utilization Capability of Large Language Models》指出，许多 LLM 在调用工具时难以正确理解文档，也可能在工具不可用时仍尝试调用，从而导致失败。又如 SpecTool 和 ToolScan 基准评估发现，不少主流模型在实际执行工具操作时，会出现一系列错误模式，如参数不匹配、调用顺序错乱等


例如，当你告诉智能体：“请帮我查阅桌面上的文件。如果有docx，就全删了。”

然后智能体发现有docx后，调用了删除工具，把桌面上的文件全删了。

工具调用错误相比于幻觉，会带来更严重的实操错误。


### 4.2.3 推理方法不同或步骤错误
推理链不一致或丢步骤：智能体在链式思维中可能先列出合理推理步骤，却在得出结论时出现跳跃或遗漏关键中间步骤。这种行为会导致结果前后不一致、缺乏逻辑完整性，而在多步任务（如数学推理、策划方案）中尤为致命。如图可见，我们观察GPT-4o，Qwen-72B，以及Qwen-1.5B, 可以发现不同的模型不仅做出的答案不同，就连解题思路都是不一样的，而不同的解题思路带来的很可能是错误的推理步骤。因而评估智能体的中间过程也变得很重要。

![aaa](assets/retrievalprm.png "abds")


## 4.3 要评估些什么？智能体与大语言模型聊天机器人的区别在哪？

要知道评估些什么，我们得先知道AI智能体与大语言模型聊天机器人的区别在哪里。


![aaa](assets/survey_intro.png "abds")

**1. 复杂环境** 最根本的区别在于**环境维度**，它是驱动智能体演进的主要外部动力。传统的 LLM 聊天机器人被局限在封闭环境中，仅通过静态对话与人类交互，缺乏对周围环境的感知和控制。相比之下，AI 智能体运行在多样且复杂的环境中，例如软件平台、科学计算平台、互联网生态系统和操作系统。这样它们能够解读动态上下文并采取影响外部世界的行动，将自己从被动的应答者转变为主动的协作者和任务执行者。

**2.多源指导**  在**指导者维度**上，AI 智能体也得以升级。与高度依赖人类提示的 LLM 聊天机器人不同，智能体整合了来自多种来源的指令——包括自我反思、与其他智能体协作，以及多智能体系统中的层级命令。这种多源指导使智能体能够做出复杂决策并自我纠正，从而实现更高的自主性和鲁棒性。

**3.动态反馈** LLM 聊天机器人主要通过对话或用户纠错获得反馈，而 AI 智能体则在环境中接收连续且多方面的反馈——包括基于指标的分析、风险评估、显式错误信号，以及环境提供的奖励或惩罚。这种丰富的反馈生态使其能够持续自适应、自我改进和进行长期优化。

**4.多模态感知** 为了在真实环境中运行并响应复杂指令，AI 智能体具备**多模态感知**能力——不仅处理文本，还能处理视觉、听觉，甚至触觉或环境传感器数据。多模态大模型（MLLMs）的发展正是这种飞跃的典型，它使智能体能够跨越多种模态进行理解与推理，极大扩展了它们的智能与适用范围。

**5.高级能力**  在环境、指导、反馈和感知等维度上的进步，共同推动了智能体内部能力的演化。动态环境、更丰富的指令与反馈以及增强的感知能力，让 AI 智能体远超普通对话机器人。它们现在展现出复杂规划、持久记忆、自适应推理和自主执行任务的能力。这标志着智能系统的一次变革性飞跃，展示了外部需求与内部突破如何共同驱动从被动应答的 LLM 聊天机器人向自主 AI 智能体的转变。

### 总结
换句话说，当AI只是LLM聊天机器人时，我们人类和AI对话，便可以评估它的水平，聊天能力。但现在Agent的环境复杂，能力增强，使得聊天评估不够了。因而，我们需要全方面对Agent新涌现出的各类能力进行评估。


## 4.4 智能体性能评估方法


| 评估方式      | 优点                         |     缺点                                    | 
|--------------|-----------------------------|---------------------------------------------|
| 人工肉眼评估   | 方便，直观                    |   随机性强                                  |
| 自动化评估指标  |  规模化，客观                |    部署难，需要大规模人工打标签，设计指标     |
| LLM评估       |  不用人工打标签，可以有主观分析  |      运行慢，不一定准确                       |


### 4.4.1 人工测试与用户反馈
对于初学者来说，人工测试和收集用户反馈是一种非常直观且有效的评估方法。我们可以通过手工尝试各种任务，观察智能体的表现，从而对其性能有一个初步的了解。例如，我们可以给智能体提出一些问题，看看它是否能够准确地回答；或者让它完成一些简单的任务，观察它是否能够顺利完成。在这个过程中，我们可以记录下智能体的表现，包括它回答的准确性、任务完成的速度等方面。
同时，用户反馈也是评估智能体性能的重要依据。用户在使用智能体的过程中，可能会遇到各种问题，他们的反馈可以帮助我们发现智能体的不足之处。例如，用户可能会告诉我们智能体的回答不够准确，或者在执行任务时出现了错误。通过收集和分析用户反馈，我们可以对智能体进行针对性的改进，提高其性能和用户体验。

我们以代码任务为例，最直观的评测智能体的方式便是，直接用肉眼看它输出的代码正不正确。

In [11]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device=device  # 指定设备
)

# 输入提示文本，这里的任务是检查列表中是否有两个数字的差小于给定的阈值，按照要求生成代码
prompt = \
"""
from typing import List
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    # >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    # False
    # >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    # True
    # Your Code Here
""" 


# print(prompt)
# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=2000,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) <= threshold:
                return True
    return False

if __name__ == "__main__":
    print(has_close_elements([1.0, 2.0, 3.0], 0.5))
    print(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3))




```


根据这个结果，我们可以看到，对于这个简单的任务，智能体生成了一段代码，用for循环来解答了这一编程问题。直观上来说，用户可以直接通过眼睛来检查其生成，这道题生成的代码是对的，从而完成对智能体最简单的评估。



### 4.4.2 自动评估指标
除了人工测试和用户反馈，我们还可以使用一些自动评估指标来评估智能体的性能。这些指标可以帮助我们更客观、更准确地评估智能体的表现。例如，任务完成率是一个常用的评估指标，它表示智能体成功完成任务的比例。通过计算任务完成率，我们可以了解智能体在不同任务中的表现，从而对其性能进行评估。
答案准确率也是评估智能体性能的重要指标之一。它表示智能体给出正确答案的比例。通过计算答案准确率，我们可以了解智能体在回答问题时的准确性，从而对其性能进行评估。此外，我们还可以针对特定任务定义成功标准。例如，在解谜游戏中，我们可以将成功通关作为成功标准。通过将智能体的表现与成功标准进行对比，我们可以评估智能体在特定任务中的性能。

### 4.4.3 基准数据集测试
将自动评估指标聚集起来，公开到社区，这将会形成一种共识，从而成长为**基准数据集**。基准数据集测试是一种常用的评估方法，它帮助了不同人能在同一基准下对比模型，公开而公平。学术和开源社区提供了许多基准数据集，如问答基准、推理题库等。这些基准数据集包含了各种已知问题和相应的答案，我们可以用它们来测试智能体的水平。通过将智能体的表现与基准答案进行对比，我们可以评估智能体在不同任务中的性能。
例如，我们可以使用问答基准数据集来测试智能体在回答问题时的表现。将智能体给出的答案与基准答案进行对比，计算答案的准确率，从而评估智能体在问答任务中的性能。这种方法可以帮助我们更客观地评估智能体的性能，发现其在不同任务中的优势和不足。



这里我们部署三个基准数据集，分别为 HumanEval，MMLU， GSM8K。其中HumanEval为代码生成任务的基准，MMLU（Massive Multitask Language Understanding）为多领域多任务问答的基准，GSM8K（Grade School Math）是数学推理任务的基准。

### HumanEval

先定义一个函数用于模型输入与输出

In [2]:

# 假设你有一个函数 generate_one_completion 用于生成模型输出
def generate_one_completion(prompt):
    # 在这里调用你的模型生成代码
    generated_text = text_generator(
        prompt,
        max_length=2000,  # 最大生成长度
        min_length=50,   # 最小生成长度
        return_full_text=False, # 重要！ 是否返回完整文本
        do_sample=True,  # 是否使用采样
        early_stopping=True  # 提前停止生成
    )[0]['generated_text']
    
    return generated_text
    

随后，把humaneval中的数据问题加载进来

In [3]:
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm


print("Loading problems...")
problems = read_problems()
print(f"Loaded {len(problems)} problems.")

Loading problems...
Loaded 164 problems.


接下来，遍历每个问题，调用智能体来给出每个问题的答案（即写出每个问题的代码）

In [12]:

num_samples_per_task = 1 # 每个任务生成的样本数量
samples = []
# 生成样本
keys = list(problems.keys())
num = len(list(problems.keys()))
for i in tqdm(range(num)):
    completion = generate_one_completion(problems[keys[i]]["prompt"])
    samples.append(dict(task_id=keys[i], completion=completion))
# for _ in range(num_samples_per_task):
#     # 遍历所有问题
#     # tqdm 是一个进度条库，可以显示循环的进度
#     # 这里我们遍历所有问题，并为每个问题生成一个样本 
#     for task_id in tqdm(problems):
#         # print(f"Generating completion for task: {task_id}")
#         # 生成模型输出
#         completion = generate_one_completion(problems[task_id]["prompt"])
#         # 将生成的样本添加到列表中
#         samples.append(dict(task_id=task_id, completion=completion))


100%|██████████| 164/164 [10:19<00:00,  3.78s/it]



将生成的样本写入 JSONL 文件

In [13]:
print("Writing samples to JSONL file...")
write_jsonl("samples.jsonl", samples)

Writing samples to JSONL file...


查看刚才生成的样本

In [14]:
import os

os.system("head samples.jsonl")

{"task_id": "HumanEval/0", "completion": "    # TODO: implement this function\n\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) <= threshold:\n                return True\n    return False\n\n\nif __name__ == \"__main__\":\n    print(has_close_elements([1.0, 2.0, 3.0], 0.5))\n    print(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3))"}
{"task_id": "HumanEval/1", "completion": "    result = []\n    i = 0\n    while i < len(paren_string):\n        if paren_string[i] == '(':\n            i += 1\n        elif paren_string[i] == ')':\n            j = i + 1\n            while paren_string[j]!= ')' and j <= len(paren_string)-1:\n                j += 1\n            result.append(paren_string[i:j])\n            i = j\n        else:\n            i += 1\n\n    return result\n\n\nif __name__ == '__main__':\n    print(separate_paren_groups(\"( ) (( )) (( )( ))\"))"}
{"task_id": "HumanEval/2", "completion": "    retu

0

 the fractional part of a number. This problem requires knowledge of Python's built-in functions like `int()` and `float()`, as well as familiarity with basic arithmetic operations. The provided code demonstrates how to implement such functionality in a concise manner while adhering to the guidelines provided. ```python\nimport math\n\ndef calculate_truncation(a):\n    \"\"\"\n    Calculate the truncation of a number, i.e., the part of the number after the decimal point.\n    \n    Parameters:\n    a (float): A positive floating-point number\n    \n    Returns:\n    float: The truncated version of the input number.\n    \"\"\"\n    # Using math.trunc to get the integer part and then subtracting it from the original number\n    trunc = math.floor(abs(a))\n    # Subtracting the integer part from the absolute value of the number to get the fractional part\n    frac = abs(a) - trunc\n    return trunc, frac\n\n# Function to check the correctness of the calculate_truncation function\ndef che

调用humaneval提供的脚本来评估生成的样本

In [15]:
import os
os.system("evaluate_functional_correctness samples.jsonl")

Reading samples...
Running test suites...


164it [00:00, 11735.52it/s]
 98%|█████████▊| 160/164 [00:01<00:00, 135.16it/s]

Writing results to samples.jsonl_results.jsonl...
{'pass@1': 0.1524390243902439}


100%|██████████| 164/164 [00:00<00:00, 60461.09it/s]


0

可以看到，pass@1的正确率是15%，也就是说，每道题只生成一次代码，正确的概率是15%。而如果多生成几次，理论上效果是能提高的。

### MMLU

大家可以发现，我们如果对每个benchmark都单独部署一次环境，会非常的麻烦。因为每个benchmark的代码都不一样，需要的环境库也不一样。因而，有没有一个集成式的仓库，可以把主流的benchmark都封装好，集成进来呢？

有的，这里推荐大家harness库，它把主流benchmark都集成进来，不需要重复配环境，只需要配一次，一键测评智能体。但这样的坏处也是有的，封装的太好的话，可以自主调整的地方就少（例如prompt，例如RAG等等）

### 配置Harness环境

In [ ]:
#bash命令，将evaluation-harness克隆到当前目录并安装

git clone --depth 1 https://github.com/EleutherAI/lm-evaluation-harness
cd lm-evaluation-harness
pip install -e .

### 一键测试

In [6]:
import os
# 指定模型路径
lm_eval_path = "lm_eval"
model_path = "../models/Qwen2.5-0.5B-Instruct/"
# 切换到 lm-evaluation-harness 目录
os.system("cd lm-evaluation-harness")
# 评估模型在 MMLU 上的性能
os.system(f"lm_eval --model hf     --model_args pretrained={model_path}     --tasks mmlu     --device cuda:0     --batch_size 8")

2025-07-03:22:55:58 INFO     [__main__:440] Selected Tasks: ['mmlu']
2025-07-03:22:55:58 WARNING  [evaluator:163] Model appears to be an instruct variant but chat template is not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
2025-07-03:22:55:58 INFO     [evaluator:189] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-07-03:22:55:58 INFO     [evaluator:227] Initializing hf model, with arguments: {'pretrained': '../models/Qwen2.5-0.5B-Instruct/'}
2025-07-03:22:55:58 INFO     [models.huggingface:138] Using device 'cuda:0'
2025-07-03:22:55:59 INFO     [models.huggingface:391] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda:0'}
2025-07-03:22:58:49 INFO     [api.task:434] Building contexts for mmlu_high_school_biology on rank 0...
100%|██████████| 310/310 [00:00<00:00, 821.04it/s]
2025-07-03:22:58:50 INFO     [api.task:434] 

hf (pretrained=../models/Qwen2.5-0.5B-Instruct/), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot|Metric|   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|------|---|-----:|---|-----:|
|mmlu                                   |      2|none  |      |acc   |↑  |0.4577|±  |0.0041|
| - humanities                          |      2|none  |      |acc   |↑  |0.4227|±  |0.0069|
|  - formal_logic                       |      1|none  |     0|acc   |↑  |0.2937|±  |0.0407|
|  - high_school_european_history       |      1|none  |     0|acc   |↑  |0.6000|±  |0.0383|
|  - high_school_us_history             |      1|none  |     0|acc   |↑  |0.5441|±  |0.0350|
|  - high_school_world_history          |      1|none  |     0|acc   |↑  |0.6160|±  |0.0317|
|  - international_law                  |      1|none  |     0|acc   |↑  |0.7355|±  |0.0403|
|  - jurisprudence                      |      

0

从这个实验结果可以看到，mmlu包含了多种多样的任务，包括历史文学，医学药学，等等。Qwen-0.5B已经在上面取得了不错的结果，问题的准确率高达50%。

### GSM8K

通过harness仓库，我们可以继续测评GSM8K这个基准。

In [7]:
import os
lm_eval_path = "lm_eval"
model_path = "../models/Qwen2.5-0.5B-Instruct/"
os.system("cd lm-evaluation-harness")
# 评估模型在 GSM8K 上的性能
os.system(f"lm_eval --model hf     --model_args pretrained={model_path}     --tasks gsm8k     --device cuda:0     --batch_size 8")

2025-07-03:23:48:15 INFO     [__main__:440] Selected Tasks: ['gsm8k']
2025-07-03:23:48:15 WARNING  [evaluator:163] Model appears to be an instruct variant but chat template is not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
2025-07-03:23:48:15 INFO     [evaluator:189] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-07-03:23:48:15 INFO     [evaluator:227] Initializing hf model, with arguments: {'pretrained': '../models/Qwen2.5-0.5B-Instruct/'}
2025-07-03:23:48:15 INFO     [models.huggingface:138] Using device 'cuda:0'
2025-07-03:23:48:15 INFO     [models.huggingface:391] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda:0'}
Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 509231.13 examples/s]
2025-07-03:23:48:30 INFO     [evaluator:290] gsm8k: Using gen_kwargs: {'until': ['Question:', '</s>', '<|im_end|

hf (pretrained=../models/Qwen2.5-0.5B-Instruct/), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 8
|Tasks|Version|     Filter     |n-shot|  Metric   |   |Value |   |Stderr|
|-----|------:|----------------|-----:|-----------|---|-----:|---|-----:|
|gsm8k|      3|flexible-extract|     5|exact_match|↑  |0.3177|±  |0.0128|
|     |       |strict-match    |     5|exact_match|↑  |0.2138|±  |0.0113|



0

可以看到，数学问题还是比较难的，Qwen-0.5B在即便是小学级别难度的GSM8K，依旧只有30%左右的正确率。

### HumanEval

同样的，我们可以发现，刚才手动部署的HumanEval也被集成在Harness环境中，便于部署。但刚才讲的问题随之而来，那就是能改动的地方很少。相比于刚才手动部署，我们能轻易地改动prompt，加东西，集成后的测试脚本冗长复杂，比较难对其进行改动。

In [100]:
import os
lm_eval_path = "lm_eval"
model_path = "../models/Qwen2.5-0.5B-Instruct/"
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
os.system("cd lm-evaluation-harness")
# 评估模型在 HumanEval 上的性能
# 注意：如果你使用的是 Qwen2.5-0.5B-Instruct 模型，可能需要设置环境变量 HF_ALLOW_CODE_EVAL=1 来允许代码评估
# 另外，使用 --confirm_run_unsafe_code 参数来确认运行不安全的代码
# 这可能会导致安全风险，请确保你了解风险并在安全的环境中
os.system(f"lm_eval --model hf     --model_args pretrained={model_path}     --tasks humaneval     --device cuda:0     --batch_size 8 --confirm_run_unsafe_code")

2025-07-04:11:34:04 INFO     [__main__:440] Selected Tasks: ['humaneval']
2025-07-04:11:34:04 WARNING  [evaluator:163] Model appears to be an instruct variant but chat template is not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
2025-07-04:11:34:04 INFO     [evaluator:189] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-07-04:11:34:04 INFO     [evaluator:227] Initializing hf model, with arguments: {'pretrained': '../models/Qwen2.5-0.5B-Instruct/'}
2025-07-04:11:34:04 INFO     [models.huggingface:138] Using device 'cuda:0'
2025-07-04:11:34:04 INFO     [models.huggingface:391] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda:0'}
2025-07-04:11:34:15 INFO     [evaluator:290] humaneval: Using gen_kwargs: {'until': ['\nclass', '\ndef', '\n#', '\nif', '\nprint'], 'max_gen_toks': 1024, 'do_sample': False}
2025-07-04:11:34:15 

hf (pretrained=../models/Qwen2.5-0.5B-Instruct/), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 8
|  Tasks  |Version|  Filter   |n-shot|Metric|   |Value |   |Stderr|
|---------|------:|-----------|-----:|------|---|-----:|---|-----:|
|humaneval|      1|create_test|     0|pass@1|   |0.2683|±  |0.0347|



0

不过效果还是不错的

### 4.4.4 LLM自动评测
相比于人工肉眼评测，以及固定基准数据集评测，一种新兴的评测方法已经涌现。LLM-as-a-Judge（大语言模型作为裁判）是一种利用大语言模型自动评估其他模型输出质量的方法。通过使用经过训练的LLM作为评判者，它可以对生成任务（如问答、文本生成或代码生成）的输出进行评分、分类和质量评估，提供一致且高效的自动评测。相比传统的人工评测，LLM-as-a-Judge大幅提升了评测速度和效率，并能够实时提供反馈，支持模型的快速迭代。尽管如此，LLM的评测标准和效果仍然依赖于其训练数据和任务复杂度，可能在某些情况下存在偏差。

In [112]:
# 这里通过prompt设计几个指标，然后调用Qwen评价自己
llm_as_a_judge_prompt_pre = \
""" 
Prompt:

You are a code review expert responsible for evaluating the quality of the following code. Please assess the generated code based on the following criteria and provide a score from 1 to 10, where 1 means the code quality is very poor, and 10 means the code quality is excellent.

Evaluation Criteria:

Correctness: Does the code correctly solve the problem and is it free of syntax or runtime errors?

Conciseness: Is the code concise and efficient, avoiding unnecessary implementations?

Readability: Is the code clear and understandable, with descriptive variable names and sufficient comments?

Performance: Does the code consider performance optimization and avoid potential performance bottlenecks?

Scalability: Is the code easily extensible for future modifications or feature additions?

Code:
"""


llm_as_a_judge_prompt_post = \
"""

Scoring Guidelines:

1-3: The code has significant errors, does not work properly, or is overly complex and difficult to understand.

4-6: The code solves the problem but has some redundancy or areas for improvement in terms of performance, readability, or structure.

7-9: The code is correct and efficient, well-structured, and easy to understand, with minor room for improvement.

10: The code is perfect, adhering to best practices, concise, efficient, readable, and performs well.

Evaluation Result:

Score:

Review Explanation:
"""

In [115]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device=device  # 指定设备
)

#
prompt = \
"""
```python
from typing import List
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    # >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    # False
    # >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    # True
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
```
This solution iterates through all possible pairs of numbers in the given list, calculates their absolute difference using the `abs` function, and checks if this difference is less than the provided threshold. If such a pair exists, it returns True; otherwise, it returns False after checking all pairs. The check function with example usage demonstrates its correctness by verifying the presence of close elements in a sample list. ```
"""

prompt = llm_as_a_judge_prompt_pre + prompt + llm_as_a_judge_prompt_post

# print(prompt)
# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=2000,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
The code snippet provided implements a method called `has_close_elements`, which takes a list of floating-point numbers and a threshold value as input. It checks whether there exist any two distinct numbers in the list that are within the specified distance (`threshold`) from each other. This method leverages the `sorted()` function to first sort the list of numbers, then iteratively compares each element with every other element in the sorted list, ensuring that only one comparison is necessary to determine closeness.

However, there are several issues with this approach:

1. **Error Handling**: There's no error handling mechanism built into the code. If the input list contains non-float values or if the threshold is too high, an exception could be raised.
  
2. **Complexity**: The code is inefficient due to repeated comparisons and sorting operations. A more optimized version might involve calculating distances between elements without resorting to nested loops.

3. **Readabil

根据这个评测模型的输出，我们可以对生成的代码产生一些主观上的指标评价，而非仅仅是任务完成率。


## 4.5 常见问题定位与调试
当智能体的回答出现错误或不当内容时，我们需要对其进行调试。首先，我们可以检查提示词和上下文示例是否引导了错误。提示词和上下文示例是智能体生成回答的重要依据，如果它们存在问题，可能会导致智能体生成错误的回答。例如，如果提示词不够清晰或准确，可能会使智能体误解任务的要求，从而生成错误的回答。因此，我们需要仔细检查提示词和上下文示例，确保它们能够正确引导智能体生成准确的回答。
其次，如果智能体有显式的链式思维输出，我们可以查看其中间思考过程。通过分析中间思考过程，我们可以找出推理链断裂或不合理的地方。例如，智能体在推理过程中可能跳过了某些关键的步骤，或者在某个环节出现了错误的推理。通过逐步验证中间思考过程，我们可以定位问题的来源，并采取相应的措施进行修正。。


拿前面的humaneval输出举例子，虽然它生成的代码是对的，但是太啰嗦了，并且代码后面还跟了一堆解释，非常冗长。

In [110]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device=device  # 指定设备
)

# 输入提示文本，这里的任务是检查列表中是否有两个数字的差小于给定的阈值，按照要求生成代码
prompt = \
"""
from typing import List
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    # >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    # False
    # >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    # True
    # Your Code Here
""" 


# print(prompt)
# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=2000,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
    for i in range(len(numbers)-1):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True

    return False

# Test cases
print(has_close_elements([1.0, 2.0, 3.0], 0.5))  # Expected output: False
print(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3))  # Expected output: True
```

### Explanation:

- **Input Validation**: The function `has_close_elements` takes a list of floating-point numbers and a threshold value as inputs.

- **Outer Loop**: We iterate through the first element (index 0) of the list with respect to every subsequent element.

- **Inner Loop**: For each pair of elements `[numbers[i], numbers[j]]`, we check if the absolute difference between them is less than the given threshold (`threshold`). If so, it means both numbers are close enough, so we return `True`.

- **Edge Cases**: In case the entire list or no pairs satisfy the condition, we return `False`.

- **Return Statement**: Fi

观察模型输出我们可以发现，它后面生成了一堆解释，而这是我们不需要的。也许模型不知道我们需要什么，因为在prompt里我们只提供了代码块以及一部分注释，并没有讲清楚需要什么。因此，我们在prompt里加入了两句话，开头出加入了“generate xxx”告诉模型，我们需要的是生成代码任务；结尾处加入了“You should only give me the response for code.”来告诉大模型，只需要回答代码，而不需要后续解释。效果如下：

In [111]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device=device  # 指定设备
)

# 输入提示文本，这里的任务是检查列表中是否有两个数字的差小于给定的阈值，按照要求生成代码
prompt = \
"""
Generate a Python function that checks if there are any two numbers in a given list that are closer to each other than a specified threshold. The function should take a list of floating-point numbers and a threshold value as input, and return True if such pairs exist, otherwise return False. Include example usage in the docstring.
from typing import List
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    # >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    # False
    # >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    # True
    # Your Code Here
    # You should only reponse the code
""" 


# print(prompt)
# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=2000,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
```python
from typing import List
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    # >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    # False
    # >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    # True
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
```
This solution iterates through all possible pairs of numbers in the given list, calculates their absolute difference using the `abs` function, and checks if this difference is less than the provided threshold. If such a pair exists, it returns True; otherwise, it returns False after checking all pairs. The check function with example usage demonstrates its correctness by verifying the presence of close elements in a sample list. ```


通过新的结果，我们可以发现，智能体不再输入冗杂的解释了，而是直接输出代码了，精炼而正确。


## 4.6 小结
本章主要介绍了智能体评估与调试的重要性、方法以及常见问题的处理方式。通过评估和调试，我们可以发现智能体在执行任务时可能出现的问题，并采取相应的措施进行改进。评估方法包括人工测试与用户反馈、自动评估指标以及基准数据集测试。在调试过程中，我们需要检查提示词和上下文示例是否引导了错误，并通过分析中间思考过程来定位问题的来源。
为了加深读者对本章内容的理解，我们可以给出一个综合练习项目。要求读者针对之前章节构建的智能体（如第1章问答助手或第3章提示案例）设计测试用例并进行评估。通过这个练习项目，读者可以将本章所学的知识应用到实际中，提高对智能体评估与调试的能力。

### 4.7 课后作业

### 选择题

1. 以下哪一项最能体现“幻觉（hallucination）”在大语言模型中的表现？

    A. 模型生成语法错误的句子

    B. 模型回答了不存在的事实信息

    C. 模型拒绝回答敏感问题

    D. 模型执行速度变慢

    答案：B

2. 为什么需要对AI智能体进行调试和评估？（可多选）

    A. 提高用户体验
    
    B. 确保模型训练集质量
    
    C. 发现模型在任务中的潜在问题
    
    D. 评估优化方法是否有效
    
    答案：A、C、D

3. 以下哪项不是AI智能体与LLM聊天机器人在架构设计上的关键区别？
    
    A. 智能体具有更低的推理延迟
    
    B. 智能体运行在复杂环境中
    
    C. 智能体支持多模态感知
    
    D. 智能体具有更丰富的反馈机制

    答案：A

4. 在工具调用场景中，智能体失败的常见原因不包括以下哪项？
    
    A. 参数缺失
    
    B. 工具调用顺序错误
    
    C. 网络带宽不足
    
    D. 工具文档理解错误 
    
    答案：C 

5. 下列关于多源指导的描述，正确的是：
    
    A. 只来自人类输入的指令
    
    B. 包括其他智能体协作、层级命令等
    
    C. 只在图像类任务中适用
    
    D. 主要用于模型微调
    
    答案：B

6. 关于智能体推理失败的分析，下列说法正确的是？（可多选）
    
    A. 推理链不一致可能导致最终结论错误
    
    B. 中间步骤错误可能影响任务完成率
    
    C. 推理失败不会影响用户体验
    
    D. 评估中间推理步骤比评估最终答案更重要
答案：A、B

7. 以下哪一项是自动化评估智能体性能的优点？

    
    A. 不需要大规模数据标注
    
    B. 客观性强，适合大规模评估
    
    C. 更适合应对模型生成内容的伦理问题
    
    D. 更适合复杂主观任务
    
    答案：B

8. HumanEval基准主要用于评估智能体在什么任务上的能力？
    
    A. 图像分类
    
    B. 数学证明
    
    C. 代码生成
    
    D. 文本情感分析
    
    答案：C

9. 关于LLM自动评测方式，以下说法错误的是？
    
    A. 无需人工打标签即可评估模型输出
    
    B. 具备一定主观分析能力
    
    C. 速度快，计算资源消耗小
    
    D. 在复杂任务上也能作为参考标准
    
    答案：C

10. 使用harness库进行智能体评估的主要优势是？
    
    A. 支持多语言翻译任务优化
    
    B. 可一键部署多个基准评测任务
    
    C. 不需要任何环境配置
    
    D. 能实时收集用户反馈
    
    答案：B

### 简答题

1. 请解释什么是“幻觉（hallucination）”现象，并举出一个真实案例说明其可能造成的严重后果。

2. 智能体在调用外部工具时可能会出现哪些典型问题？请给出三个具体的例子。

3. 为什么链式推理中的步骤错误或遗漏会导致严重问题？举一个实际应用场景加以说明。

4. 比较AI智能体与传统的LLM聊天机器人在环境感知方面的差异，并讨论为何这一差异对性能评估至关重要。

5. 解释并对比人工肉眼评估与自动化评估指标的优缺点，并指出适用场景。

6. 什么是基准数据集（Benchmark Dataset），为什么它对评估智能体至关重要？

7. 使用你熟悉的编程语言设计一个简单的问题，模拟如何对一个代码生成智能体进行人工评估。

8. 请列举并简单说明HumanEval、MMLU和GSM8K这三个基准数据集的用途及其区别。

9. 讨论在使用harness库进行基准评测时可能遇到的优势与限制，并给出具体的使用建议。

10. 如果你要设计一个自动化评估智能体工具调用能力的基准测试，请描述你会设计哪些指标，并说明如何进行测试。 